In [1]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1920,
        'height': 1080,
        'scroll': True,
})
%config InlineBackend.figure_format = 'retina'

# Week 04 (Wednesday), AST 8581 / PHYS 8581 / CSCI 8581: Big Data in Astrophysics

### Michael Coughlin <cough052@umn.edu>, Michael Steinbach <stei0062@umn.edu>, Nico Adams adams900@umn.edu


With contributions totally ripped off from Gautham Narayan (UIUC)

* Simple MC with uniform sampling of parameter space **does not solve curse of dimensionality (too many useless samples in low likelihood region)** 
* What if, instead of sampling the parameter space uniformly, you could sample the posterior directly
    * Possible outcomes would be **simulated with a frequency proportional to the probability**
    
<table>
    <tr>
        <td><img src="figures/Likelihood_Surface.png" width=100%></td>
    </tr>
</table>

There are a couple of approaches to this:
 
### 1. Rejection sampling
For this method, we need to define an *envelope function* which everywhere exceeds the target PDF, $p(x)$, and can be sampled. Let this be $Ag(x)$ where $A$ is a scaling factor and $g(x)$ is a PDF we know.

Then the algorithm is
```
while we want more samples
    draw a random value for x from some distribution g in the variable x
    draw u from Uniform(0,1)
    if u <= p(x)/(A*g(x)), keep the sample x
    otherwise, reject x
```

Visually, this corresponds to drawing points that uniformly fill in the space under $p(x)$.

<table>
    <tr>
        <td><img src="figures/mc1_rejection.png" width=100%></td>
    </tr>
</table>
Courtesy: Phil Marshall


# This also does not solve the problem posed by the curse of dimensionality 

# But this approach is general and works for any function 

The second approach you've already seen and used a lot:

### 2. The Inverse Transform 

The definition of the CDF (and it's inverse, $F^{-1}$, the quantile function - i.e. `ppf()` in `scipy.stats`) 

$F(x) = P(X \leq x) = \int_{-\infty}^x p(x')\,dx'$

By this definition, quantiles of $X$ are uniformly distributed on [0,1]. If $F^{-1}$ is easy to evaluate, we can use this straightforwardly:

```
draw u from Uniform(0,1)
compute x = F_inverse(u)
```



<table>
    <tr>
        <td>PDF<img src="figures/mc1_invtrans0.png" width=100%></td>
        <td></td>
        <td>CDF<img src="figures/mc1_invtrans1.png" width=100%></td>
    </tr>
</table>

Courtesy: Phil Marshall

# In class warm-up: The PIT with exponential distributions:

This distribution has $p(x)=\lambda e^{-\lambda x}$ and $F(x)=1-e^{-\lambda x}$ for $x\geq0$.

The quantile function is, therefore, $F^{-1}(P) = -\ln(1-P)/\lambda$.

In [3]:
# Here's some code for the inverse tranform

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


def inv_trans_demo(x, lam):
    hist = plt.hist(x, bins=50)
    xs = np.linspace(0.0, 10.0/lam, 100)
    pdf = lam * np.exp(-lam*xs)
    pdfline = plt.plot(xs, pdf, 'r', lw=2)
    plt.xlabel(r'x', fontsize=22)
    plt.ylabel(r'P(x)', fontsize=22);

* for lam = 1., draw a 1000 random samples from a uniform distribution between 0-1 (np.random.rand should do)
* use inverse CDF $F^{-1}(P)$ to convert to samples from a exponential distribution
* then use the demo code to histogram your samples and overplot the distribution.

In [1]:
# YOUR CODE HERE


* This does solve the problem posed by the curse of dimensionality 

* No sample is rejected! 

* The con is that you have to know what p(x) looks like in advance

* If p(x) is your posterior, then you not only need to be able to solve for it analytically (including the evidence - the denominator of Bayes' theorem) but then you've got to figure out how to invert it... even harder.
    
* There is a place for the PIT, but we started down this road because our functions weren't generally going to be nice, so lets deal with rejection sampling some more

# We want a couple of properties

* We want to sample the full distribution 
* We want the frequency of samples between $x$ and $x+dx$ to be proportional to $p(x)dx$

What if, instead of drawing i.i.d samples, we drew samples such that they are correlated with each other.

You have already seen examples of processes where samples are correlated with each other - Brownian motion/random walks/Wiener processes - all of these are examples of **stochastic** processes
<table>
    <tr>
        <td><img src="figures/Wiener_process_3d.png" width=100%></td>
    </tr>
</table>

 What if we chose our samples to be correlated in a very specific way:
   * decide how many samples you want
   * start somewhere - we'll call our current position in k-dimensional parameter space $x$
   * while you want samples:
       * perturb $x$ to $x'$ by some random vector drawn from a fixed distribution   
           * i.e. the samples are correlated
       * evaluate your function at $x'$ and $x$
       * if the function is higher at $x'$ than at $x$ 
           * then yay! Accept it, and set the position $x'$ to the current position $x$
       * else if the function is lower at $x'$ than at $x$
           * well maybe that's bad, or maybe we're just unlucky and there's good samples to be had near here
           * How do we decide? Well let's draw a random number and check if our function ratio is better or worse
               * if it's better, accept and set the position $x'$ to the current position $x$
               * else reject and update the current position to be the same 
       * stick the current position after you did this into a list of samples 
    
This sequence/list of all accepted samples is a **chain**

This is just our **rejection sampling** strategy (recall):

```
while we want more samples
    draw a random value for x from some distribution g in the variable x
    draw u from Uniform(0,1)
    if u <= p(x)/(A*g(x)), keep the sample x
    otherwise, reject x
```

# Markov Chains:

If we can construct a sequence of samples/chain this way, it will be **ergodic** - i.e. given enough time, the full distribution will be sampled

This chain is from some n-dimensional parameter space, with a distribution that is asymptotically proportional to $p(x)$. 

The constant of proportionality is not important in the first class of problems we will look at. 

In model comparison problems, the proportionality constant must be known. We've glossed over that so far, so we will blithely push forward.

With our particular strategy, every n+1 th position on the chain depends **only** on the nth position:

<table>
    <tr>
        <td><img src="figures/MarkovChain.png" width=100%></td>
    </tr>
</table>

Chains that have this property are called **Markov Chains**.

The **state space** of this stochastic process is the set of all possible values

The particular algorithm for generating new samples (more properly detailed in HW4) is called **Metropolis-Hastings** after the folks that came up with it.

In summary, the Metropolis-Hastings algorithm consists of these steps:

1. given $x$ and $T(x'|x)$, draw a proposed value for $x'$

2. compute acceptance probability $p_{\rm acc}(x,x')$.

3. draw a random number between 0 and 1 from a uniform distribution; if it smaller than $p_{\rm acc}(x,x')$, then accept $x'$.

4. if $x'$ is accepted added it to the chain, if not, add $x'$ to the chain.

See below for why it works!

How far should we step (small steps in parameter space or large). This impacts the efficiency of the process but not if we will reach equilibrium.

<table>
    <tr>
        <td><img src="figures/sampling.png" width=100%></td>
    </tr>
</table>


# WHY DOES Metropolis-Hastings work?

This process is NOT **stationary**. 

#### Why does it work?
The probability of an arbitrary point from such a chain being located at $x'$ is (marginalizing over the possible immediately preceding points)

## $$p(x') = \int dx \, p(x) \, T(x'|x)$$

where $T(x'|x)$ is the transition probability of a step from $x$ to $x'$.

If we have detailed balance, 

## $$p(x)T(x'|x) = p(x')T(x|x')$$

rearranging:

## $$ \frac{T(x'|x)}{T(x|x')} = \frac{p(x')}{p(x)} $$

The basic trick to connect this with rejection sampling is to break the transition into two steps:
1. A proposal, g(x'| x)
and 
2. Acceptance ratio, A(x'|x)

i.e. 

## $$ T(x'|x) = A(x'|x) g(x'| x) $$ 

rearranging again :

## $$ \frac{A(x'|x)}{A(x|x')} = \frac{p(x')g(x|x')}{p(x)g(x'|x) }$$

Notice that the probability of accepting a step  (once it's proposed) is

## $$A(x',x) = \mathrm{min}\left[1, \frac{p(x')g(x|x')}{p(x)g(x'|x)}\right]$$

Let's look again at the requirement of detailed balance

> the probability of being at $x$ and moving to $y$ must equal the probability of being at $x'$ and moving to $x$

The first of these is $p(x)g(x'|x)A(x',x)$, where

* $p(x)$ is the posterior density (probability of *being* at $x$, if we're sampling $P$ properly)

* $g(x'|x)$ is the proposal distribution (probability of attempting a move to $x'$ from $x$)

* $A(x',x)$ is the probability of accepting the proposed move

With this definition of $A$, detailed balance is automatically satisfied!

## $$p(x)g(x'|x)A(x',x) \equiv p(x')g(x|x')A(x,x')$$

Note that **even if a step is rejected, we still keep a sample** (the original state, without moving). The difficulty of finding a temptingly better point is important information!

## Speeding things up

Broadly speaking, we can try to
1. tailor algorithms to specific classes of PDF
2. look for ways to make the general samplers more intelligent

We can also use different samplers for different subsets of parameters - the only rule is that every parameter must get updated somehow.

## Gibbs Sampling
is a specialization of Metropolis-Hastings:
* Instead of making a general proposal in all dimensions, we cycle through the parameters proposing changes to **one at a time**
* A proposal for $\theta_i$ is into the **fully conditional posterior** $p(\theta_i|\theta_{-i},x)$, where $-i$ means all subscripts other than $i$.

Gibbs sampling:
```
while we want more samples
    propose theta1 | theta2, theta3, ..., data
    accept/reject theta1
    propose theta2 | theta1, theta3, ..., data
    accept/reject theta2
    ...
```

### Example: normal PDF

25 Metropolis iterations (left) vs. 25 Gibbs transitions (right)

Color goes blue$\rightarrow$red with time (step number)

<table>
    <tr>
        <td>
            <img src="figures/mc2_metro.png" width=100%>
        </td>
        <td></td>
        <td>
            <img src="figures/mc2_gibbs.png" width=100%>
        </td>
    </tr>
</table>

In general, this is not obviously an improvement to proposing changes to all $\theta$ simultaneously.

Why is a random drunk walking in one specific direction at a time better than just taking a random step???

Something interesting happens if the fully conditional likelihood and prior are **conjugate** 

For some likelihood functions, if you choose a certain prior, *the posterior ends up being in the same distribution as the prior.* Such a prior then is called a **Conjugate Prior.**

i.e. 

# $$ P(\theta) \mathrm{\; such\; that\; } P(\theta|D) = P(\theta) $$


i.e. we know the conditional posterior exactly!

If we use independent samples of the conditional posterior as proposals, then the Metropolis-Hastings acceptance ratio becomes

## $$\frac{p(x')g(x|x')}{p(x)g(x'|x)} = \frac{p(x')p(x)}{p(x)p(x')} = 1$$

**and every proposal is automatically accepted!**

Conjugate Gibbs Sampling:
```
while we want more samples
    draw th1 from p(th1|th2,th3,...,data)
    draw th2 from p(th2|th1,th3,...,data)
    ...
```

* Beta posterior

    * Beta prior * Bernoulli likelihood → Beta posterior
    * Beta prior * Binomial likelihood → Beta posterior
    * Beta prior * Negative Binomial likelihood → Beta posterior
    * Beta prior * Geometric likelihood → Beta posterior

* Gamma posterior
   * Gamma prior * Poisson likelihood → Gamma posterior
   * Gamma prior * Exponential likelihood → Gamma posterior

* Normal posterior
    * Normal prior * Normal likelihood (mean) → Normal posterior

Gibbs Sampling Pros:
* No cycles "wasted" on rejected proposals
* No pesky tuning of the proposal scale

Gibbs Sampling Cons:    
* Only works for conjugate or partially conjugate models (hence must choose conjugate priors)
* Occasionally still slower than proposing multi-parameter Metropolis updates (e.g. when degeneracies are strong)

Some multiple modes:
<table>
    <tr>
        <td>
            Some spectral model<br>
            <img src="figures/mc2_multimode_eg2.png" width=100%>
        </td>
        <td></td>
        <td>
            The eggbox function<br>
            <img src="figures/mc2_eggbox.png" width=100%>
        </td>
    </tr>
</table>

In these cases, when we were talking about optimizers (as opposed to samplers) that used gradient descent, we found that local optimizers got stuck

<table>
    <tr>
        <td>
            Some spectral model<br>
            <img src="figures/global_vs_local.png" width=100%>
        </td>
    </tr>
</table>

We talked about **simulated annealing** and **basin hopping** as examples of *global optimizers* that can get out of these local minima.

With MH, you'd have to make a large step size, but that has a major downside - your acceptance ratio goes down and your autocorrelation time goes up.

So, it's reasonable to ask if there's an analog for MCMC.

## Tempering
Consider the function $[p(x)]^{1/T}$, where $p(x)$ is the target PDF.
* We say a chain sampling this function has temperature $T$. For $T>1$, $p^{1/T}$ is smoothed out compared with $p$.
    * This is the same sort of thing that we saw with **simulated annealing** - start with sampling at a high temperature and then gradually lower the temperature with some schedule
* This allows chains to move among multiple peaks more easily.
* Of course, we're only actually interested in $T=1$...

### Parallel tempering

With parallel tempering, we run one chain with $T=1$ and several more chains with $T>1$. A modified Metropolis-Hastings update occasionally allows the chains to exchange positions, giving the $T=1$ chain a mechanism for sampling regions of parameter space it might otherwise have low probability of proposing. Samples from the $T=1$ chain can be used for inference.

## Coping with multiple modes

Multiple, well separated posterior modes are a serious challenge for many samplers.
* In general, the only way to discover that they exist is by exploring the parameter space with many widely dispersed chains.
* To do inference, our chains need to be able to efficiently transition between modes - so far the most reliable general method we've seen for this is parallel tempering. 

## Implementations of other samplers that are commonly used in research:

1. Gibbs Sampling ([pymc3](https://docs.pymc.io/))
2. Parallel Tempering (now in [ptemcee](https://pypi.org/project/ptemcee/))
3. Hamiltonian Monte Carlo/No U-turn Sampling (also [pymc3](https://docs.pymc.io/))

And this is also a sampler, but is not actually a Markov Chain method (Monte Carlo is logically separate from Markov Chain Monte Carlo)
4. Nested Sampling ([dynesty](https://dynesty.readthedocs.io/en/latest/index.html) or [Multinest](https://johannesbuchner.github.io/PyMultiNest/))


We can't cover all of these, but you should have a good idea on how to start with any of them by now, and which one to pick for a particular problem.

## In Class Exercise: think

Recall the ugly PDF features we were motivated by, namely strong/nonlinear degeneracies and multiple modes.
For each of the methods above, do you expect an improvement compared with standard Metropolis in these situations. 

Why and for which methods?

<table>
    <tr>
        <td>
            <img src="figures/mc2_rosenbrock.png" width=80%>
        </td>
        <td></td>
        <td>
            <img src="figures/mc2_eggbox.png" width=80%>
        </td>
    </tr>
</table>

# In Class Exercise:

Here are some simple functions. Use my MH sampling code to sample them, and compare to what you get on a grid of x, y

In [2]:
import numpy as np

def Rosenbrock_lnP(x, y, a=1.0, b=100.0):
    if y < 0.0: return -np.inf
    return -( (a-x)**2 + b*(y-x**2)**2 )

def eggbox_lnP(x, y):
    return (2.0 + np.cos(0.5*x)*np.cos(0.5*y))**3

def sphshell_lnP(x, y, s=0.1):
    return -(np.sqrt(x**2+y**2) - 1)**2/(2.0*s**2)


In [8]:
def metropolis_hastings(p, x0, sigma, *args, nsamp=1000):
    
    ndim = len(x0)
    try:
        test_val = p(*x0, *args)
        if not np.isfinite(test_val):
            raise ValueError('Function at starting position is not finite')
        
        if test_val == 0.:
             raise ValueError('Function at starting position must be non-zero')
                
    except Exception as e:
        message = f'{e}\nCannot initialize sampler at this position'
        raise ValueError(message)
    
        
    # we need something to save the samples we want
    samples = np.zeros((nsamp, ndim))    

    x = np.array(x0)
    sigma = np.array(sigma)
    
    # the position and step size arrays had better be the same 
    assert x.shape == sigma.shape, 'Shape of x and shape of sigma must be the same'
    
    # while we want more samples
    for i in range(nsamp):

        # now we adjust the initial position a little
        # instead of explictly definition g(x|x') and g(x'|x)
        # we can recognize that a Gaussian is a stationary kernel
        # as we discussed in class, this is nice because 
        # all that matters is the absolute difference between x' and x
        # and if that's the case, then g(x'|x) = g(x|x')
        x_prime = x + sigma*np.random.randn(ndim)


        if np.random.rand() < (p(*x_prime, *args) / p(*x, *args)):
            x = x_prime
            
        # we save the sample to the chain
        samples[i] = x
    return samples




def mh_demo(p, x0, sigma, nsamp=1000):
    samples = metropolis_hastings(p, x0, sigma, nsamp=nsamp)

    
    # create some axes
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))
    ax0, ax1, ax2 = axs
    
    # plot all the data in grey
    ax0.scatter(samples[:, 0], samples[:, 1], color='grey', alpha=0.1, marker='.')
    ax1.plot(samples[:, 0], marker='.', color='grey', alpha=0.1, ls='-')
    ax2.plot(samples[:, 1], marker='.', color='grey', alpha=0.1, ls='-')
    return fig
    

In [2]:
# Your code here